<a href="https://colab.research.google.com/github/MattNandavong/TDP_ChatBot/blob/main/Test_model_swin_fineTuneBertQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import json

! pip install datasets transformers accelerate torch
import torch

from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer, DataCollatorWithPadding


In [2]:
from transformers import BertTokenizer

def preprocess_data(data, tokenizer):
    preprocessed_data = []
    for entry in data:
      for cont in entry['context']:
        context = entry['context']
        for qa in entry['qas']:
            question = qa['question']
            answer_text = qa['answers'][0]['text']
            answer_start = qa['answers'][0]['answer_start']
            answer_end = qa['answers'][0]['answer_end']

            # Tokenize context and question
            tokenized_inputs = tokenizer(context, question, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

            # Extract tokenized inputs
            input_ids = tokenized_inputs['input_ids'].squeeze(0)
            attention_mask = tokenized_inputs['attention_mask'].squeeze(0)
            token_type_ids = tokenized_inputs['token_type_ids'].squeeze(0)

            preprocessed_data.append({
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'token_type_ids': token_type_ids,
                'start_positions': answer_start,
                'end_positions': answer_end,
                'context': context,
                'question': question,
                'answer': answer_text
            })
    return preprocessed_data


Provided dataset

In [4]:
with open('test_qa.json', 'r') as data:
  dataset = json.load(data)
  # print(dataset)

load tokenizer

In [5]:
class CustomQADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.data[idx]['input_ids']),
            'attention_mask': torch.tensor(self.data[idx]['attention_mask']),
            'token_type_ids': torch.tensor(self.data[idx]['token_type_ids']),
            'start_positions': torch.tensor(self.data[idx]['start_positions']),
            'end_positions': torch.tensor(self.data[idx]['end_positions']),
            'context': self.data[idx]['context'],
            'question': self.data[idx]['question'],
            'answer': self.data[idx]['answer']
        }

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
processed_data = preprocess_data(dataset, tokenizer)
# print(processed_data)
print(processed_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Preprocessing dataset

In [7]:
custom_dataset = CustomQADataset(processed_data)
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="QA_model6-test3",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=2e-5,
    save_steps=500,
    save_total_limit=2,
    logging_dir="logs",
    logging_steps=100,
    weight_decay=0.01,
    push_to_hub = True
)

In [9]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = custom_dataset,
    tokenizer=tokenizer,
    data_collator = data_collator
)

In [18]:
# print(Trainer.train.__doc__)

In [10]:
print(processed_data[0].keys())


dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'start_positions', 'end_positions', 'context', 'question', 'answer'])


In [11]:
trainer.save_model()
trainer.train()


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

<ipython-input-5-ad3693dc0a9c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.data[idx]['input_ids']),
<ipython-input-5-ad3693dc0a9c>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.data[idx]['attention_mask']),
<ipython-input-5-ad3693dc0a9c>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(self.data[idx]['token_type_ids']),


Step,Training Loss
100,1.230500
200,0.144700
300,0.049300
400,0.020600
500,0.007600
600,0.002900
700,0.001100
800,0.000100
900,0.015900
1000,0.004600


<ipython-input-5-ad3693dc0a9c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.data[idx]['input_ids']),
<ipython-input-5-ad3693dc0a9c>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.data[idx]['attention_mask']),
<ipython-input-5-ad3693dc0a9c>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(self.data[idx]['token_type_ids']),
<ipython-input-5-ad3693dc0a9c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceT

TrainOutput(global_step=2730, training_loss=0.05456623486314082, metrics={'train_runtime': 1682.9093, 'train_samples_per_second': 12.972, 'train_steps_per_second': 1.622, 'total_flos': 2.027460234514637e+16, 'train_loss': 0.05456623486314082, 'epoch': 3.0})